In [1]:
from typing import List
import torch
import torch.nn as nn
import numpy as np

## 1. Data Preparation

In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [3]:
dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
dataset_train = dataset["train"]
dataset_val = dataset["validation"]
dataset_test = dataset["test"]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
MAX_LEN = 113

In [6]:
from torch.utils.data import Dataset

class NER_Dataset(Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.tokens = dataset["tokens"]
        self.labels = dataset["ner_tags"]
        self.tokenizer = tokenizer
        self.max_len = MAX_LEN 

    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, idx):
        input_token = self.tokens[idx]
        label_token = self.labels[idx]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)

        input_ids = self.pad_and_truncate(input_token, pad_id= self.tokenizer.pad_token_id)
        labels = self.pad_and_truncate(label_token, pad_id=0)
        attention_mask =  self.pad_and_truncate(attention_mask, pad_id=0)

        return {
            "input_ids": torch.as_tensor(input_ids), 
            "labels": torch.as_tensor(labels),
            "attention_mask": torch.as_tensor(attention_mask)
            }
    
    def pad_and_truncate(self, inputs: List[int], pad_id: int): 
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return padded_inputs

In [7]:
train_set = NER_Dataset(dataset_train, tokenizer)
val_set = NER_Dataset(dataset_val, tokenizer)
test_set = NER_Dataset(dataset_test, tokenizer)

## 2. Model

In [8]:
from transformers import AutoModelForTokenClassification

label2id = {
    'O': 0, 
    'B-PER': 1, 
    'I-PER': 2, 
    'B-ORG': 3, 
    'I-ORG': 4, 
    'B-LOC': 5, 
    'I-LOC': 6, 
    'B-MISC': 7, 
    'I-MISC': 8,
}
id2label = {v:k for k, v in label2id.items()}

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
model

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

## 3. Training

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != 0
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="adamw_torch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.016941,0.841102
2,No log,0.013130,0.873416
3,0.019000,0.013988,0.877717
4,0.019000,0.015334,0.883297
5,0.005600,0.015944,0.888062
6,0.005600,0.016704,0.887830
7,0.003700,0.018384,0.889341
8,0.003700,0.017889,0.892479
9,0.003700,0.018470,0.893177
10,0.002600,0.018812,0.893874


TrainOutput(global_step=2200, training_loss=0.007211010984399102, metrics={'train_runtime': 1564.1509, 'train_samples_per_second': 89.768, 'train_steps_per_second': 1.407, 'total_flos': 8097818252099580.0, 'train_loss': 0.007211010984399102, 'epoch': 10.0})

## 4. Testing

In [11]:
trainer.evaluate(test_set)

{'eval_loss': 0.016628410667181015,
 'eval_accuracy': 0.846694622594968,
 'eval_runtime': 11.8618,
 'eval_samples_per_second': 291.102,
 'eval_steps_per_second': 4.552,
 'epoch': 10.0}

## 5. Test sample

In [19]:
# test_sentence = "[UNK] rejects [UNK] call to boycott [UNK] lamb."
test_sentence = "France won the World Cup in Russia in 2018"
inputs = tokenizer(test_sentence, return_tensors="pt", add_special_tokens=False) # Use the function as training data
inputs

{'input_ids': tensor([[1699, 1281, 1103, 1291, 1635, 1107, 2733, 1107, 1857]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
for key, value in inputs.items():
    inputs[key] = inputs[key].to("cuda")

In [21]:
outputs = model(**inputs)
outputs.logits.shape

torch.Size([1, 9, 9])

In [22]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()
preds

array([5, 0, 0, 7, 8, 0, 5, 0, 0])

In [23]:
pred_tags = ""

for pred in preds:
    pred_tags += id2label[pred] + " "

pred_tags

'B-LOC O O B-MISC I-MISC O B-LOC O O '